In [19]:
from tensorflow.keras import layers
from tensorflow.keras import models

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LearningRateScheduler
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

import optuna

In [20]:
Features=pd.read_csv("../data/data_proccesed.csv")
X = Features.iloc[: ,:-1].values
Y = Features['labels'].values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

def custom_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.02))
scheduler = LearningRateScheduler(custom_scheduler)

model_save = ModelCheckpoint(r"../model/best_model.h5", save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

normalizer = layers.Normalization()
normalizer.adapt(X)

In [21]:
def objective(trial):
    # Define the search space
    num_neurons_dense = trial.suggest_int('num_neurons_dense', 128, 1024, log=True)
    num_neurons_conv_1 = trial.suggest_int('num_neurons_conv_1', 128, 1024, log=True)
    num_neurons_conv_2 = trial.suggest_int('num_neurons_conv_2', 128, 1024, log=True)
    num_neurons_conv_3 = trial.suggest_int('num_neurons_conv_3', 128, 1024, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.4)
    batch_size = trial.suggest_categorical('batch_size', [32, 64])

    model = models.Sequential([
        layers.Input(shape=(x_train.shape[1],)), 
        normalizer,
        layers.Reshape((x_train.shape[1], 1)), 
        layers.Conv1D(num_neurons_conv_1, 1, activation='relu'),
        layers.Conv1D(num_neurons_conv_2, 1, activation='relu'),
        layers.MaxPooling1D(),
        layers.Conv1D(num_neurons_conv_3, 1, activation='relu'),
        layers.MaxPooling1D(),
        layers.Flatten(),
        layers.Dropout(dropout_rate),
        layers.Dense(num_neurons_dense, activation='relu'),
        layers.Dense(9, activation='softmax'),
    ])
    

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )
    
    # Fit the model
    history = model.fit(
        x_train, y_train,
        epochs=40, batch_size=batch_size,
        validation_data=(x_test, y_test),
        callbacks=[scheduler, model_save, early_stopping],
        verbose=0
    )
    
    # Return the validation loss
    return history.history['val_loss'][-1]

# Perform optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)

# Print best parameters
print('Best trial:')
trial = study.best_trial
print('  Value: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2024-05-10 02:08:12,477] A new study created in memory with name: no-name-d2fb8b14-904d-48b0-85c7-ca0916887753
c:\Users\Kuba\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
[I 2024-05-10 02:09:29,658] Trial 0 finished with value: 0.6935650706291199 and parameters: {'num_neurons_dense': 212, 'num_neurons_conv_1': 204, 'num_neurons_conv_2': 952, 'num_neurons_conv_3': 634, 'dropout_rate': 0.2903076765530297, 'optimizer': 'adam', 'batch_size': 32}. Best is trial 0 with value: 0.6935650706291199.
[I 2024-05-10 02:10:07,311] Trial 1 finished with value: 0.7715939283370972 and parameters: {'num_neurons_dense': 385, 'num_neurons_conv_1': 808, 'num_neurons_conv_2': 258, 'num_neurons_conv_3': 227, 'dropout_rate': 0.170325439

KeyboardInterrupt: 